In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import math
import re

# =============================================
# 네이버 쇼핑 댓글 수집기
# =============================================

# 1. 경로 설정
fx_path = r"C:\py_temp\2025-10-17-12-41-57-네이버쇼핑 크롤러 제출용\2025-10-17-12-41-57-네이버쇼핑 크롤러 제출용.xlsx"
drv_path = r"C:\py_temp\chromedriver.exe"

# 2. 수집할 상품 개수 입력
product_cnt = int(input("몇 개 상품의 댓글을 수집할까요?: ").strip() or "1")

# 3. 엑셀에서 상품 목록 읽기
df_products = pd.read_excel(fx_path, sheet_name=0)

# 4. 결과를 담을 리스트 준비
review_product_num = []    # 상품번호
review_product_name = []   # 상품이름
review_id = []             # 작성자ID
review_date = []           # 작성날짜
review_content = []        # 댓글내용

# 5. 드라이버 초기화
svc = Service(drv_path)
driver = webdriver.Chrome(service=svc)
wait = WebDriverWait(driver, 10)
driver.maximize_window()

print("\n===== 댓글 수집 시작 =====\n")

# =============================================
# 메인 크롤링 루프
# =============================================

try:
    for i in range(min(product_cnt, len(df_products))):
        
        # 6. 10개마다 드라이버 재시작 (메모리 정리 - 중요!)
        if i > 0 and i % 10 == 0:
            print(f"\n[{i}번째 상품] 드라이버 재시작 중...\n")
            driver.quit()
            time.sleep(2)
            driver = webdriver.Chrome(service=svc)
            wait = WebDriverWait(driver, 10)
            driver.maximize_window()
        
        # 7. 상품 정보 가져오기
        product_link = df_products.loc[i, '상품링크']
        product_name = df_products.loc[i, '상품이름']
        product_num  = df_products.loc[i, '번호']
        
        if pd.isna(product_link):
            continue
        
        print(f"[{i+1}/{product_cnt}] {str(product_name)[:40]}")
        
        # 8. 상품 페이지로 이동
        driver.get(str(product_link).strip())
        time.sleep(1.5)
        
        # 9. 리뷰 탭 클릭
        try:
            review_tab = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "a[data-name='REVIEW']")
            ))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", review_tab)
            time.sleep(0.3)
            review_tab.click()
            time.sleep(1)
            print("  → 리뷰 탭 클릭 성공")
        except:
            print("  ⚠️ 리뷰 탭을 찾을 수 없음 → 다음 상품으로")
            continue
        
        # 10. 리뷰 총 개수 확인 → 페이지 수 계산
        try:
            review_tab_element = driver.find_element(By.CSS_SELECTOR, "a[data-name='REVIEW']")
            review_count_text = review_tab_element.find_element(By.CSS_SELECTOR, "span.YzoPmthSiB").text
            total_reviews = int(review_count_text.replace(',', ''))
            max_pages = min(math.ceil(total_reviews / 20), 200)  # 최대 200페이지
        except:
            max_pages = 1
        
        print(f"  → 총 {max_pages}페이지 예상")
        
        # 11. 페이지별 댓글 수집
        for page_no in range(1, max_pages + 1):
            print(f"  → 페이지 {page_no} 수집 중...")
            
            # 12. 2페이지부터는 페이지 번호 클릭
            if page_no > 1:
                try:
                    page_btn = driver.find_element(By.XPATH, 
                        f"//a[normalize-space()='{page_no}'] | //button[normalize-space()='{page_no}']"
                    )
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", page_btn)
                    time.sleep(0.3)
                    page_btn.click()
                    time.sleep(1)
                except:
                    print(f"  ⚠️ {page_no}페이지 버튼을 찾을 수 없음 → 종료")
                    break
            
            # 13. 리뷰 카드 찾기
            try:
                review_cards = driver.find_elements(By.CSS_SELECTOR, 
                    "li.PxsZltB5tV._nlog_click._nlog_impression_element"
                )
                print(f"     ({len(review_cards)}개 리뷰 발견)")
            except:
                print("  ⚠️ 리뷰를 찾을 수 없음")
                continue
            
            # 14. 각 리뷰에서 정보 추출
            for card in review_cards:
                try:
                    # 작성자 ID
                    user_id = card.find_element(By.CSS_SELECTOR, "strong.MX91DFZo2F").text.strip()
                except:
                    user_id = "익명"
                
                try:
                    # 작성날짜
                    date = card.find_element(By.CSS_SELECTOR, "span.MX91DFZo2F").text.strip()
                except:
                    date = ""
                
                try:
                    # 더보기 버튼 있으면 클릭
                    more_btn = card.find_element(By.XPATH, ".//button[contains(., '더보기')]")
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", more_btn)
                    time.sleep(0.1)
                    more_btn.click()
                    time.sleep(0.2)
                except:
                    pass  # 더보기 버튼이 없으면 그냥 진행
                
                try:
                    # 댓글 내용 (여러 방법 시도)
                    try:
                        text = card.find_element(By.CSS_SELECTOR, "div.KqJ8Qqw082 span.MX91DFZo2F").text.strip()
                    except:
                        try:
                            text = card.find_element(By.CSS_SELECTOR, "div.KqJ8Qqw082").text.strip()
                        except:
                            text = card.find_element(By.CSS_SELECTOR, "span.MX91DFZo2F").text.strip()
                except:
                    text = ""
                
                # 15. 댓글 내용이 있으면 리스트에 추가
                if text:
                    review_product_num.append(product_num)
                    review_product_name.append(product_name)
                    review_id.append(user_id)
                    review_date.append(date)
                    review_content.append(text)
        
        # 현재까지 수집된 개수 출력
        print(f"  ✅ 현재까지 총 {len(review_id)}개 댓글 수집됨\n")

except KeyboardInterrupt:
    print("\n\n⚠️ 사용자가 중단했습니다.")

except Exception as e:
    print(f"\n\n⚠️ 예상치 못한 에러 발생: {e}")

finally:
    # 16. 드라이버 종료
    driver.quit()
    
    # 17. 결과를 DataFrame으로 만들어 엑셀에 저장
    if review_id:
        df_result = pd.DataFrame({
            '상품번호': review_product_num,
            '상품이름': review_product_name,
            '작성자ID': review_id,
            '작성날짜': review_date,
            '댓글내용': review_content
        })
        
        # 기존 엑셀 파일에 새 시트로 추가
        with pd.ExcelWriter(fx_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
            df_result.to_excel(writer, sheet_name='댓글목록', index=False)
        
        print(f"\n💾 총 {len(df_result)}개 댓글 저장 완료!")
        print(f"   → {fx_path}")
        print(f"   → Sheet2: '댓글목록'\n")
    else:
        print("\n⚠️ 수집된 댓글이 없습니다.\n")

print("===== 작업 완료 =====")

몇 개 상품의 댓글을 수집할까요?:  100



===== 댓글 수집 시작 =====

[1/100] [플라스틱아일랜드]BEIGE배색카라 기본핏 셔츠 블라우스 (PR7WB16
  → 리뷰 탭 클릭 성공
  → 총 1페이지 예상
  → 페이지 1 수집 중...
     (2개 리뷰 발견)
  ✅ 현재까지 총 2개 댓글 수집됨

[2/100] [레이어드set] 카라 셔츠 뷔스티에 플리츠 롱 원피스
  → 리뷰 탭 클릭 성공
  → 총 2페이지 예상
  → 페이지 1 수집 중...
     (20개 리뷰 발견)
  → 페이지 2 수집 중...
     (1개 리뷰 발견)
  ✅ 현재까지 총 23개 댓글 수집됨

[3/100] 스파오 콜라보 긴팔 파자마 모음 (산리오캐릭터즈,포켓몬 등)
  → 리뷰 탭 클릭 성공
  → 총 200페이지 예상
  → 페이지 1 수집 중...
     (0개 리뷰 발견)
  → 페이지 2 수집 중...
     (20개 리뷰 발견)
  → 페이지 3 수집 중...
     (20개 리뷰 발견)
  → 페이지 4 수집 중...
     (20개 리뷰 발견)
  → 페이지 5 수집 중...
     (20개 리뷰 발견)
  → 페이지 6 수집 중...
     (20개 리뷰 발견)
  → 페이지 7 수집 중...
     (20개 리뷰 발견)
  → 페이지 8 수집 중...
     (20개 리뷰 발견)
  → 페이지 9 수집 중...
     (20개 리뷰 발견)
  → 페이지 10 수집 중...
     (20개 리뷰 발견)
  → 페이지 11 수집 중...
     (20개 리뷰 발견)
  → 페이지 12 수집 중...
     (20개 리뷰 발견)
  → 페이지 13 수집 중...
     (20개 리뷰 발견)
  → 페이지 14 수집 중...
     (20개 리뷰 발견)
  → 페이지 15 수집 중...
     (20개 리뷰 발견)
  → 페이지 16 수집 중...
     (20개 리뷰 발견)
  → 페이지 17 수집 중...
     (20개 리뷰 발견)
  → 페이지 